In [2]:
import json
import os
from typing import List, Tuple

from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import PromptTemplate
from deepeval.models.base_model import DeepEvalBaseLLM
from langchain_community.llms.ollama import Ollama
import asyncio


from dotenv import load_dotenv

from embeddings import *
from llms import *
# from query_simple_rag import *

load_dotenv()


/Volumes/data1/Programming/predictiv_chatbot/eva/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.1.6, however version 1.1.9 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


True

In [ ]:
ollama_llm_mistral = Ollama(
    model="llama3.1:8b",
    base_url="http://192.168.180.131:11434"
)

class OllamaMistralModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        # Sends prompt to the Ollama instance and retrieves the response
        response = self.model.invoke(prompt)
        return response["text"]  # Adjust if the output is structured differently

    async def a_generate(self, prompt: str) -> str:
        # Asynchronous wrapper around the generate method
        return self.generate(prompt)
    
    async def a_generate_raw_response(self, prompt: str, logprobs=False, top_logprobs=0) -> (str, float):
        # Erstelle die logische Anfrage an das Modell
        response = self.generate(prompt)  # Verwende die gleiche Logik wie in `generate`
        cost = 0.01  # Beispiel für Kostenberechnung (kann angepasst werden)
        return response, cost

    def get_model_name(self):
        return "Mistral Nemo via Ollama"
    
mistral_model = OllamaMistralModel(ollama_llm_mistral)

def create_deep_eval_test_cases(
    questions: List[str],
    gt_answers: List[str],
    generated_answers: List[str],
    retrieved_documents: List[str]
) -> List[LLMTestCase]:
    """
    Create a list of LLMTestCase objects for evaluation.

    Args:
        questions (List[str]): List of input questions.
        gt_answers (List[str]): List of ground truth answers.
        generated_answers (List[str]): List of generated answers.
        retrieved_documents (List[str]): List of retrieved documents.

    Returns:
        List[LLMTestCase]: List of LLMTestCase objects.
    """
    return [
        LLMTestCase(
            input=question,
            expected_output=gt_answer,
            actual_output=generated_answer,
            retrieval_context=retrieved_document
        )
        for question, gt_answer, generated_answer, retrieved_document in zip(
            questions, gt_answers, generated_answers, retrieved_documents
        )
    ]



# Define evaluation metrics
correctness_metric = GEval(
    name="Correctness",
    model=mistral_model,
    evaluation_params=[
        LLMTestCaseParams.EXPECTED_OUTPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT
    ],
    evaluation_steps=[
        "Determine whether the actual output is factually correct based on the expected output."
    ],
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model=mistral_model,
    include_reason=False
)

relevance_metric = ContextualRelevancyMetric(
    threshold=1,
    model=mistral_model,
    include_reason=True
)

def evaluate_rag(chunks_query_retriever, num_questions: int = 5) -> None:
    mistral_model, llama_model = initialize_models()
    
    # Load questions and answers from JSON file
    q_a_file_name = os.path.join('data', 'json', 'q_a.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    # Generate answers and retrieve documents for each question
    for question in questions:
        collection_name='document_embeddings_v10'
        persist_directory='chroma_db_v4'
        
        context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        filter_context = create_context_for_llm(context)
        retrieved_documents.append([filter_context])
      
        result = ask_llm_with_ollama(question, filter_context, mistral_model)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
            # Create test cases and evaluate
    test_cases = create_deep_eval_test_cases(questions, ground_truth_answers, generated_answers, retrieved_documents)
    evaluate(
        test_cases=test_cases,
        metrics=[correctness_metric, faithfulness_metric, relevance_metric]
    )
    
evaluate_rag("hallo")
        

In [ ]:
from query_simple_rag import *
def create_deep_eval_test_cases(
    questions: List[str],
    gt_answers: List[str],
    generated_answers: List[str],
    retrieved_documents: List[str]
) -> List[LLMTestCase]:
    """
    Create a list of LLMTestCase objects for evaluation.

    Args:
        questions (List[str]): List of input questions.
        gt_answers (List[str]): List of ground truth answers.
        generated_answers (List[str]): List of generated answers.
        retrieved_documents (List[str]): List of retrieved documents.

    Returns:
        List[LLMTestCase]: List of LLMTestCase objects.
    """
    return [
        LLMTestCase(
            input=question,
            expected_output=gt_answer,
            actual_output=generated_answer,
            retrieval_context=retrieved_document
        )
        for question, gt_answer, generated_answer, retrieved_document in zip(
            questions, gt_answers, generated_answers, retrieved_documents
        )
    ]


# Define evaluation metrics
correctness_metric = GEval(
    name="Correctness",
    model="gpt-3.5-turbo",
    evaluation_params=[
        LLMTestCaseParams.EXPECTED_OUTPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT
    ],
    evaluation_steps=[
        "Determine whether the actual output is factually correct based on the expected output."
    ],
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model="gpt-3.5-turbo",
    include_reason=False
)

relevance_metric = ContextualRelevancyMetric(
    threshold=1,
    model="gpt-3.5-turbo",
    include_reason=True
)

def evaluate_rag(chunks_query_retriever, num_questions: int = 5) -> None:
    mistral_model, llama_model = initialize_models()
    
    # Load questions and answers from JSON file
    q_a_file_name = os.path.join('data', 'json', 'q_a.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    # Generate answers and retrieve documents for each question
    for question in questions:
        collection_name='document_embeddings_v20'
        persist_directory='chroma_db_v4'
        
        context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        filter_context = create_context_for_llm(context)
        retrieved_documents.append([filter_context])
      
        result = ask_llm_with_ollama(question, filter_context, mistral_model)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
            # Create test cases and evaluate
    test_cases = create_deep_eval_test_cases(questions, ground_truth_answers, generated_answers, retrieved_documents)
    evaluate(
        test_cases=test_cases,
        metrics=[correctness_metric, faithfulness_metric, relevance_metric]
    )
    
evaluate_rag("hallo")
        

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM

api_key = os.getenv('use_auth_token')

class Mistral7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cpu" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        loop = asyncio.get_running_loop()
        return await loop.run_in_executor(None, self.generate, prompt)

    def get_model_name(self):
        return "Mistral 7B"


model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", use_auth_token=api_key)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", use_auth_token=api_key)

mistral_7b = Mistral7B(model=model, tokenizer=tokenizer)
print(mistral_7b.generate("Write me a joke"))

In [ ]:
ollama_llm_mistral = Ollama(
    model="mistral-nemo",
    base_url="http://192.168.180.131:11434"
)

class OllamaMistralModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        # Sends prompt to the Ollama instance and retrieves the response
        response = self.model.invoke(prompt)
        return response["text"]  # Adjust if the output is structured differently

    async def a_generate(self, prompt: str) -> str:
        # Asynchronous wrapper around the generate method
        return self.generate(prompt)
    
    async def a_generate_raw_response(self, prompt: str, logprobs=False, top_logprobs=0) -> (str, float):
        # Erstelle die logische Anfrage an das Modell
        response = self.generate(prompt)  # Verwende die gleiche Logik wie in `generate`
        cost = 0.01  # Beispiel für Kostenberechnung (kann angepasst werden)
        return response, cost

    def get_model_name(self):
        return "Mistral Nemo via Ollama"
    
mistral_model = OllamaMistralModel(ollama_llm_mistral)

def create_deep_eval_test_cases(
    questions: List[str],
    gt_answers: List[str],
    generated_answers: List[str],
    retrieved_documents: List[str]
) -> List[LLMTestCase]:
    """
    Create a list of LLMTestCase objects for evaluation.

    Args:
        questions (List[str]): List of input questions.
        gt_answers (List[str]): List of ground truth answers.
        generated_answers (List[str]): List of generated answers.
        retrieved_documents (List[str]): List of retrieved documents.

    Returns:
        List[LLMTestCase]: List of LLMTestCase objects.
    """
    return [
        LLMTestCase(
            input=question,
            expected_output=gt_answer,
            actual_output=generated_answer,
            retrieval_context=retrieved_document
        )
        for question, gt_answer, generated_answer, retrieved_document in zip(
            questions, gt_answers, generated_answers, retrieved_documents
        )
    ]



# Define evaluation metrics
correctness_metric = GEval(
    name="Correctness",
    model=mistral_model,
    evaluation_params=[
        LLMTestCaseParams.EXPECTED_OUTPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT
    ],
    evaluation_steps=[
        "Determine whether the actual output is factually correct based on the expected output."
    ],
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model=mistral_model,
    include_reason=False
)

relevance_metric = ContextualRelevancyMetric(
    threshold=1,
    model=mistral_model,
    include_reason=True
)

def evaluate_rag(chunks_query_retriever, num_questions: int = 5) -> None:
    mistral_model, llama_model = initialize_models()
    
    # Load questions and answers from JSON file
    q_a_file_name = os.path.join('data', 'json', 'q_a.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    # Generate answers and retrieve documents for each question
    for question in questions:
        collection_name='document_embeddings_v10'
        persist_directory='chroma_db_v4'
        
        context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        filter_context = create_context_for_llm(context)
        retrieved_documents.append([filter_context])
      
        result = ask_llm_with_ollama(question, filter_context, mistral_model)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
            # Create test cases and evaluate
    test_cases = create_deep_eval_test_cases(questions, ground_truth_answers, generated_answers, retrieved_documents)
    evaluate(
        test_cases=test_cases,
        metrics=[faithfulness_metric]
        # metrics=[correctness_metric, faithfulness_metric, relevance_metric]

    )
    
evaluate_rag("hallo")
        

In [5]:
def get_answers_for_gpt(num_questions: int = 5) -> None:
    
    mistral_model, llama_model = initialize_models()
    
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["gt_answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []

    for question in questions:
        collection_name='document_embeddings_v10'
        persist_directory='chroma_db_v4'
        
        context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        filter_context = create_context_for_llm(context)
        retrieved_documents.append([filter_context])
      
        result = ask_llm_with_ollama(question, filter_context, mistral_model)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
        # Rückgabe der generierten Antworten und der abgerufenen Dokumente
    return {
        "generated_answers": generated_answers,
        "retrieved_documents": retrieved_documents
    }
    
# result = get_answers_for_gpt(num_questions=5)
# print(result["generated_answers"])  # Gibt die generierten Antworten aus
# print(result["retrieved_documents"])  # Gibt die abgerufenen Dokumente aus


def create_combined_json(num_questions: int = 5, output_file: str = 'combined_output.json') -> None:
    # Lade die Fragen, GT-Antworten und generiere die Antworten sowie die abgerufenen Dokumente
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    # Holen Sie die Antworten und die Dokumente von der Funktion `get_answers_for_gpt`
    gpt_results = get_answers_for_gpt(num_questions)
    
    generated_answers = gpt_results["generated_answers"]
    retrieved_documents = gpt_results["retrieved_documents"]

    # Erstelle eine Liste, die die neue Struktur der Daten enthält
    combined_data = []
    
    for idx, qa in enumerate(q_a[:num_questions]):
        # Kombinieren Sie die originalen Daten mit den generierten Antworten und Dokumenten
        combined_entry = {
            "question": qa["question"],
            "gt_answer": qa["gt_answer"],
            "generated_answers": generated_answers[idx],
            "retrieved_documents": retrieved_documents[idx]
        }
        combined_data.append(combined_entry)

    # Speichern Sie das kombinierte Ergebnis in einer neuen JSON-Datei
    output_file_path = os.path.join('data', 'json', output_file)
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        json.dump(combined_data, outfile, indent=4, ensure_ascii=False)
    
    print(f"Die Datei wurde erfolgreich erstellt und in {output_file_path} gespeichert.")
    
create_combined_json(num_questions=5, output_file='combined_output.json')

Embedding erstellt
Basierend auf dem bereitgestellten Kontext kann ich die Telefonnummer der FES aus Dokument 2 entnehmen. Die Telefonnummer der FES Frankfurter Entsorgungs- und Service GmbH lautet: 0800 2008007-0.
Embedding erstellt
Um Ihr Altglas ordnungsgemäß zu entsorgen, sollten Sie es in einem der blauen Altglascontainer an öffentlichen Plätzen oder auf Wertstoffhöfen depositären. Stellen Sie dabei sicher, dass das Glas sauber ist und Flaschen sowie Deckelgläser nach den Farben Weiß, Grün und Braun getrennt werden. Blaues oder buntes Glas sollte immer zum Grünglas gegeben werden. Stark verschmutzte Gläser gehören hingegen in den Restmüll. Die FES Frankfurter Entsorgungs- und Service GmbH bietet weitere Informationen und Services regarding der Abfallentsorgung an.
Embedding erstellt
Plastikverpackungen können in der gelben Verpackungstonne entsorgt werden. Diese steht gemäß dem bereitgestellten Kontext 14-tägig am Wohnhaus zur Verfügung und ist für Kunststoff- und Metallverpackung

In [4]:
"""
unteresuche die folgende frage aus der JSON auf die folgenden aspekte, gebe mir scores wie von deepeval aus (zwischen 0 und 1): 
Correctness Evaluation
Faithfulness Evaluation
Contextual Relevancy Evaluation

Das Schema sieht wie folgt aus und bedeutet: 
"question": Die orginale Frage
"gt_answer": Die erwartete Antwort
"Antwort des LLM": Antwort des LLMs
"Kontext aus RAG:" Der Kontext aus den Dokumenten vom RAG

Gebe nur die Scores zur jeweiligen Frage aus. Gebe am Ende den Durchschnitt aus.
"""

'\nunteresuche die folgende frage aus der JSON auf die folgenden aspekte, gebe mir scores wie von deepeval aus (zwischen 0 und 1): \nCorrectness Evaluation\nFaithfulness Evaluation\nContextual Relevancy Evaluation\n\nDas Schema sieht wie folgt aus und bedeutet: \n"question": Die orginale Frage\n"gt_answer": Die erwartete Antwort\n"Antwort des LLM": Antwort des LLMs\n"Kontext aus RAG:" Der Kontext aus den Dokumenten vom RAG\n\n'

In [3]:
# simple RAG
from query_simple_rag import *

def get_answers_for_gpt_1(num_questions: int = 5) -> None:
    
    mistral_model, llama_model = initialize_models()
    
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["gt_answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    for question in questions:
        collection_name='document_embeddings_v20'
        persist_directory='chroma_db_v4'
        
        context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        filter_context = create_context_for_llm(context)
        retrieved_documents.append([filter_context])
      
        result = ask_llm_with_ollama(question, filter_context, mistral_model)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
        # Rückgabe der generierten Antworten und der abgerufenen Dokumente
    return {
        "generated_answers": generated_answers,
        "retrieved_documents": retrieved_documents
    }
    
# result = get_answers_for_gpt(num_questions=5)
# print(result["generated_answers"])  # Gibt die generierten Antworten aus
# print(result["retrieved_documents"])  # Gibt die abgerufenen Dokumente aus


def create_combined_json_1(num_questions: int = 21, output_file: str = 'combined_output_simple.json') -> None:
    # Lade die Fragen, GT-Antworten und generiere die Antworten sowie die abgerufenen Dokumente
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    # Holen Sie die Antworten und die Dokumente von der Funktion `get_answers_for_gpt`
    gpt_results = get_answers_for_gpt_1(num_questions)
    
    generated_answers = gpt_results["generated_answers"]
    retrieved_documents = gpt_results["retrieved_documents"]

    # Erstelle eine Liste, die die neue Struktur der Daten enthält
    combined_data = []
    
    for idx, qa in enumerate(q_a[:num_questions]):
        # Kombinieren Sie die originalen Daten mit den generierten Antworten und Dokumenten
        combined_entry = {
            "question": qa["question"],
            "gt_answer": qa["gt_answer"],
            "generated_answers": generated_answers[idx],
            "retrieved_documents": retrieved_documents[idx]
        }
        combined_data.append(combined_entry)

    # Speichern Sie das kombinierte Ergebnis in einer neuen JSON-Datei
    output_file_path = os.path.join('data', 'json', output_file)
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        json.dump(combined_data, outfile, indent=4, ensure_ascii=False)
    
    print(f"Die Datei wurde erfolgreich erstellt und in {output_file_path} gespeichert.")
    
create_combined_json_1(num_questions=21, output_file='combined_output_simple.json')

Embedding erstellt
Um Altglas zu entsorgen, müssen Sie es nach Farben sortieren: Weiß, Grün und Braun getrennt. Blaues oder buntes Glas gehört zum Grünglas. Entsorgen Sie das saubere Altglas in den dafür vorgesehenen Containern an öffentlichen Plätzen. Vermeiden Sie es, verschmutzte Gläser, Flachglas wie Spiegel und Scheiben, Trinkgläser und Blumenvasen, Porzellan, Keramik und Glühbirnen in die Altglascontainer zu geben. Diese Gegenstände gehören nicht ins Altglas und können den Recyclingprozess stören oder müssen anderweitig entsorgt werden.
Embedding erstellt
Die gelbe Tonne ist der richtige Ort für Ihre Plastikverpackungen. Bitte stellen Sie sicher, dass Sie die Verpackungen von anderen Materialien trennen und nicht ineinander stopfen, damit sie besser recycelt werden können. Kunststofftüten sollten aus Pappkartons herausgenommen werden und Deckel von Joghurtbechern abgemacht werden.
Es ist wichtig zu beachten, dass stark verschmutzte Verpackungen wie Fast-Food-Verpackungen nicht in

In [5]:
# Context RAG
from context_enrichment_rag import *

def get_answers_for_gpt_2(num_questions: int = 5) -> None:
    
    mistral_model, llama_model = initialize_models()
    
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["gt_answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    for question in questions:
        collection_name='document_embeddings_v20'
        persist_directory='chroma_db_v4'
        
        context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        context_list = get_context_list(context, collection_name, persist_directory)
        filter_context = create_context_for_llm(context_list)
        retrieved_documents.append([filter_context])
      
        result = ask_llm_with_ollama(question, filter_context, mistral_model)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
        # Rückgabe der generierten Antworten und der abgerufenen Dokumente
    return {
        "generated_answers": generated_answers,
        "retrieved_documents": retrieved_documents
    }
    
# result = get_answers_for_gpt(num_questions=5)
# print(result["generated_answers"])  # Gibt die generierten Antworten aus
# print(result["retrieved_documents"])  # Gibt die abgerufenen Dokumente aus


def create_combined_json_2(num_questions: int = 21, output_file: str = 'combined_output_context.json') -> None:
    # Lade die Fragen, GT-Antworten und generiere die Antworten sowie die abgerufenen Dokumente
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    # Holen Sie die Antworten und die Dokumente von der Funktion `get_answers_for_gpt`
    gpt_results = get_answers_for_gpt_2(num_questions)
    
    generated_answers = gpt_results["generated_answers"]
    retrieved_documents = gpt_results["retrieved_documents"]

    # Erstelle eine Liste, die die neue Struktur der Daten enthält
    combined_data = []
    
    for idx, qa in enumerate(q_a[:num_questions]):
        # Kombinieren Sie die originalen Daten mit den generierten Antworten und Dokumenten
        combined_entry = {
            "question": qa["question"],
            "gt_answer": qa["gt_answer"],
            "generated_answers": generated_answers[idx],
            "retrieved_documents": retrieved_documents[idx]
        }
        combined_data.append(combined_entry)

    # Speichern Sie das kombinierte Ergebnis in einer neuen JSON-Datei
    output_file_path = os.path.join('data', 'json', output_file)
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        json.dump(combined_data, outfile, indent=4, ensure_ascii=False)
    
    print(f"Die Datei wurde erfolgreich erstellt und in {output_file_path} gespeichert.")
    
create_combined_json_2(num_questions=21, output_file='combined_output_context.json')

Embedding erstellt
Um Ihr Altglas richtig zu entsorgen, sollten Sie die verschiedenen Glasfarben trennen. Weiße, grüne und braune Flaschen und Deckelgläser werden jeweils in separaten Containern gesammelt. Blaues oder buntes Glas wird hingegen zum Grünglas gegeben. Stellen Sie sicher, dass das Glas sauber ist und keine Reste von Nahrungsmitteln oder Flüssigkeiten enthalten sind. Vermeiden Sie es, Glühbirnen, Spiegel, Trinkgläser oder Porzellan in den Altglascontainer zu werfen, da sie das Recycling stören können.
Embedding erstellt
Um Ihre Plastikverpackungen ordnungsgemäß zu entsorgen, werfen Sie sie in die gelbe Verpackungstonne. Stellen Sie sicher, dass die Verpackungen leer sind und nicht stark verschmutzt sind. Kunststoffverpackungen wie Folien, Becher und Styropor sowie Verbundverpackungen aus Materialmix wie Getränkekartons und Milchtüten gehören in diese Tonne.
Die gelbe Verpackungstonne wird alle 14 Tage geleert.
Embedding erstellt
Die Telefonnummer der FES Frankfurter Entsorg

In [6]:
#  Query Transform simple RAG
from query_transformation_simple_rag import *

def get_answers_for_gpt_3(num_questions: int = 5) -> None:
    
    mistral_model, llama_model = initialize_models()
    
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["gt_answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    for question in questions:
        collection_name='document_embeddings_v20'
        persist_directory='chroma_db_v4'
        
        query_transform_string = start_query_transform(question)
        liste = string_in_liste_aufteilen(query_transform_string)
        multi_filtert_context_result = dokumente_ausgeben(liste)
        # print(multi_filtert_context_result)
        # context = get_similar_documents_from_chroma(collection_name, persist_directory, question)
        # filter_context = create_context_for_llm(context)
        retrieved_documents.append([multi_filtert_context_result])
      
        result = ask_llm_with_multi_context(question, multi_filtert_context_result)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
        # Rückgabe der generierten Antworten und der abgerufenen Dokumente
    return {
        "generated_answers": generated_answers,
        "retrieved_documents": retrieved_documents
    }
    
# result = get_answers_for_gpt(num_questions=5)
# print(result["generated_answers"])  # Gibt die generierten Antworten aus
# print(result["retrieved_documents"])  # Gibt die abgerufenen Dokumente aus


def create_combined_json_3(num_questions: int = 21, output_file: str = 'combined_output_transform_simple.json') -> None:
    # Lade die Fragen, GT-Antworten und generiere die Antworten sowie die abgerufenen Dokumente
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    # Holen Sie die Antworten und die Dokumente von der Funktion `get_answers_for_gpt`
    gpt_results = get_answers_for_gpt_3(num_questions)
    
    generated_answers = gpt_results["generated_answers"]
    retrieved_documents = gpt_results["retrieved_documents"]

    # Erstelle eine Liste, die die neue Struktur der Daten enthält
    combined_data = []
    
    for idx, qa in enumerate(q_a[:num_questions]):
        # Kombinieren Sie die originalen Daten mit den generierten Antworten und Dokumenten
        combined_entry = {
            "question": qa["question"],
            "gt_answer": qa["gt_answer"],
            "generated_answers": generated_answers[idx],
            "retrieved_documents": retrieved_documents[idx]
        }
        combined_data.append(combined_entry)

    # Speichern Sie das kombinierte Ergebnis in einer neuen JSON-Datei
    output_file_path = os.path.join('data', 'json', output_file)
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        json.dump(combined_data, outfile, indent=4, ensure_ascii=False)
    
    print(f"Die Datei wurde erfolgreich erstellt und in {output_file_path} gespeichert.")
    
create_combined_json_3(num_questions=21, output_file='combined_output_transform_simple.json')

Embedding erstellt
Um Ihr Altglas korrekt zu entsorgen, sortieren Sie es zunächst nach Farben: Weiß, Grün, Braun. Blaues oder buntes Glas gehört zum Grünglas. Stellen Sie das saubere und leere Glas in den dafür vorgesehenen Altglascontainer an öffentlichen Plätzen. Vermeiden Sie bitte, Flachglas wie Spiegel und Scheiben, Trinkgläser, Porzellan, Keramik, Glühbirnen oder stark verschmutzte Gläser in den Altglascontainer zu geben. Diese gehören in den Restmüll.
Embedding erstellt
Plastikverpackungen gehören in den Restmüll oder bei schadstoffhaltiger Füllung (z.B. Farben, Lacke, Flüssigdünger) in die Schadstoffsammlung. Bitte Trennung von Materialien und keine Verstopfung.
Embedding erstellt
Embedding erstellt
Die Telefonnummer der FES Frankfurter Entsorgungs- und Service GmbH lautet 0800 2008007-0.
Embedding erstellt
Embedding erstellt
Im Altglascontainer gehören saubere Flaschen und Deckelgläser nach den Farben Weiß, Grün und Braun getrennt. Blaues oder buntes Glas sollte immer zum Grün

In [7]:
# Context RAG
from query_transformation_context_rag import *

def get_answers_for_gpt_4(num_questions: int = 5) -> None:
    
    mistral_model, llama_model = initialize_models()
    
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["gt_answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []
    
    for question in questions:
        collection_name='document_embeddings_v20'
        persist_directory='chroma_db_v4'
        
        query_transform_string = start_query_transform(question)
        liste = string_in_liste_aufteilen(query_transform_string)
        multi_filtert_context_result = dokumente_ausgeben(liste)

        retrieved_documents.append([multi_filtert_context_result])
      
        # result = ask_llm_with_ollama(question, multi_filtert_context_result, mistral_model)
        result = ask_llm_with_multi_context(question, multi_filtert_context_result)
        print(result)  # Fügen Sie dies hinzu, um zu sehen, was zurückgegeben wird
        # Füge den String in ein Dictionary-ähnliches Format um
        # modified_result = {"answer": result}
        generated_answers.append(result)
        
        # Rückgabe der generierten Antworten und der abgerufenen Dokumente
    return {
        "generated_answers": generated_answers,
        "retrieved_documents": retrieved_documents
    }
    
# result = get_answers_for_gpt(num_questions=5)
# print(result["generated_answers"])  # Gibt die generierten Antworten aus
# print(result["retrieved_documents"])  # Gibt die abgerufenen Dokumente aus


def create_combined_json_4(num_questions: int = 21, output_file: str = 'combined_output_transform_context.json') -> None:
    # Lade die Fragen, GT-Antworten und generiere die Antworten sowie die abgerufenen Dokumente
    q_a_file_name = os.path.join('data', 'json', 'q_a_for_gpt.json')
    
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    # Holen Sie die Antworten und die Dokumente von der Funktion `get_answers_for_gpt`
    gpt_results = get_answers_for_gpt_4(num_questions)
    
    generated_answers = gpt_results["generated_answers"]
    retrieved_documents = gpt_results["retrieved_documents"]

    # Erstelle eine Liste, die die neue Struktur der Daten enthält
    combined_data = []
    
    for idx, qa in enumerate(q_a[:num_questions]):
        # Kombinieren Sie die originalen Daten mit den generierten Antworten und Dokumenten
        combined_entry = {
            "question": qa["question"],
            "gt_answer": qa["gt_answer"],
            "generated_answers": generated_answers[idx],
            "retrieved_documents": retrieved_documents[idx]
        }
        combined_data.append(combined_entry)

    # Speichern Sie das kombinierte Ergebnis in einer neuen JSON-Datei
    output_file_path = os.path.join('data', 'json', output_file)
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        json.dump(combined_data, outfile, indent=4, ensure_ascii=False)
    
    print(f"Die Datei wurde erfolgreich erstellt und in {output_file_path} gespeichert.")
    
create_combined_json_4(num_questions=21, output_file='combined_output_transform_context.json')

Embedding erstellt
Um Ihr Altglas ordnungsgemäß zu entsorgen, sollten Sie es nach Farben sortieren: Weiß, Grün und Braun. Blaues oder buntes Glas wird zum Grünglas gegeben. Stellen Sie sicher, dass das Glas sauber ist und keine Flachglasgegenstände wie Spiegel oder Scheiben sowie stark verschmutzte Gläser eingeworfen werden. Verwenden Sie dafür Altglascontainer an öffentlichen Plätzen.
Embedding erstellt
Plastikverpackungen gehören in die gelbe Tonne oder den gelben Sack. Bitte stellen Sie sicher, dass die Verpackungen leer sind und Trennung von verschiedenen Materialien erfolgt (z.B. Deckel ab bei Kunststoffbechern). Beachten Sie jedoch auch eventuelle regionale Besonderheiten wie Wertstofftonnen. Ihre kommunale Abfallberatung kann Ihnen weiterhelfen.
Embedding erstellt
Die Telefonnummer der FES Frankfurter Entsorgungs- und Service GmbH ist 0800 2008007-0.
Embedding erstellt
Embedding erstellt
Glascontainer:
- Weißes, grünes und braunes Glas getrennt
- Konservengläser

Altkleidersamml